In [1]:
import pytorch_lightning as pl
from models.inpainting_model import InpaintingModel
from models.flow_model import FlowModel
from models.occlusion_model import OcclusionModel
from models.flow_occ_model import FlowOccModel
# flow occ model should be ['simple', 'flowoccnets', 'flowoccnetc', 'pwoc', 'flowoccnet']
# optical flow model should be ['simple', 'flownets', 'flownetc', 'pwc', 'flownet']
# occlusion model should be ['simple', 'occnets', 'occnetc']
hparams = dict(learning_rate=1e-3, batch_size=1, image_size=(512, 1024), model='flowoccnet')
root = r'C:\Users\caodo\Desktop\TUM\Praktikum Visual Computing\project\data'
model = FlowOccModel(root=root, hparams=hparams)

In [2]:
max_epochs = 5

trainer = pl.Trainer(
    max_epochs=max_epochs,
    gpus=0
)

trainer.fit(model)

GPU available: True, used: False
No environment variable for node rank defined. Set as 0.

    | Name                                               | Type                 | Params
----------------------------------------------------------------------------------------
0   | model                                              | FlowOccNet           | 6 M   
1   | model.correlation_layer                            | CostVolumeLayer      | 0     
2   | model.feature_pyramid_network                      | FeaturePyramidNet    | 1 M   
3   | model.feature_pyramid_network.layer1               | DoubleConv           | 2 K   
4   | model.feature_pyramid_network.layer1.double_conv   | Sequential           | 2 K   
5   | model.feature_pyramid_network.layer1.double_conv.0 | Conv2d               | 432   
6   | model.feature_pyramid_network.layer1.double_conv.1 | BatchNorm2d          | 32    
7   | model.feature_pyramid_network.layer1.double_conv.2 | LeakyReLU            | 0     
8   | model.feature

C:\Users\caodo\anaconda3\lib\site-packages\torch\nn\functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Val-Loss: 14.3299


Detected KeyboardInterrupt, attempting graceful shutdown...


1